In [ ]:
import logging

from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import convert_files_to_docs
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

In [ ]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True)

doc_dir = "./api/data/build_your_first_question_answering_system/"


In [ ]:
all_docs = convert_files_to_docs(dir_path=doc_dir)

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

In [ ]:
document_store.write_documents(docs)

In [ ]:
retriever = BM25Retriever(document_store=document_store)

reader = FARMReader(model_name_or_path='etalab-ia/camembert-base-squadFR-fquad-piaf', use_gpu=True)

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
prediction = pipe.run(
    query="Quelle est la banque?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)

In [ ]:
prediction

In [ ]:
prediction['answers']